In [ ]:
## Task-1

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import json
from torch.utils.data import DataLoader
from tqdm import tqdm

# ---------------------------------------
# STEP 1: Load Pretrained ResNet-34 Model
# ---------------------------------------
print("Loading pretrained ResNet-34 model...")
resnet_model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
resnet_model.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model.to(device)

# ---------------------------------------
# STEP 2: Define Image Preprocessing
# ---------------------------------------
print("Setting up image transforms...")
imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

image_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

# ---------------------------------------
# STEP 3: Load the 100-Class ImageNet Subset
# ---------------------------------------
print("Loading test dataset...")
dataset_path = "/content/drive/MyDrive/TestDataSet"  # Update this path if needed
test_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=image_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ---------------------------------------
# STEP 4: Load ImageNet Class Index Mapping
# ---------------------------------------
print("Loading imagenet_class_index.json...")
with open("/content/drive/MyDrive/TestDataSet/imagenet_class_index.json", "r") as f:
    class_index_data = json.load(f)

# Build: synset (folder name) → ImageNet class index
synset_to_index = {entry[0]: int(key) for key, entry in class_index_data.items()}

# Map dataset class index (0-99) → actual ImageNet class index
folder_to_index = test_dataset.class_to_idx
dataset_idx_to_imagenet_idx = {
    folder_to_index[synset]: synset_to_index[synset]
    for synset in folder_to_index
}

# ---------------------------------------
# STEP 5: Run Evaluation (Top-1 & Top-5)
# ---------------------------------------
print("Evaluating the model...")
top1_hits = 0
top5_hits = 0
total_samples = 0

with torch.no_grad():
    for images, labels in tqdm(test_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Make predictions
        outputs = resnet_model(images)
        _, top5_predictions = outputs.topk(5, dim=1)

        # Map ground truth to real ImageNet indices
        true_labels = torch.tensor(
            [dataset_idx_to_imagenet_idx[int(label)] for label in labels]
        ).to(device)

        # Top-1 Accuracy
        top1_predictions = top5_predictions[:, 0]
        top1_hits += (top1_predictions == true_labels).sum().item()

        # Top-5 Accuracy
        top5_hits += sum([true_labels[i] in top5_predictions[i] for i in range(len(true_labels))])

        total_samples += labels.size(0)

# ---------------------------------------
# STEP 6: Show Final Results
# ---------------------------------------
top1_accuracy = 100 * top1_hits / total_samples
top5_accuracy = 100 * top5_hits / total_samples

print(f"\nTop-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")


Loading pretrained ResNet-34 model...
Setting up image transforms...
Loading test dataset...
Loading imagenet_class_index.json...
Evaluating the model...


100%|██████████| 16/16 [00:02<00:00,  5.65it/s]


Top-1 Accuracy: 76.00%
Top-5 Accuracy: 94.20%


In [ ]:
## Task-2

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import json
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# FGSM attack parameters
epsilon = 0.02
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Standard ImageNet normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define preprocessing and un-normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

unnormalize = transforms.Normalize(
    mean=[-m/s for m, s in zip(mean, std)],
    std=[1/s for s in std]
)

# Load the test dataset
dataset_path = "/content/drive/MyDrive/TestDataSet"
test_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load ImageNet class mapping (official format: index → [synset, label])
with open("/content/drive/MyDrive/TestDataSet/imagenet_class_index.json", "r") as f:
    class_map = json.load(f)

# Map synset (folder name) → class index
synset_to_index = {v[0]: int(k) for k, v in class_map.items()}
folder_name_to_index = test_dataset.class_to_idx
index_mapping = {
    folder_name_to_index[synset]: synset_to_index[synset]
    for synset in folder_name_to_index
}

# Load the pretrained ResNet-34 model
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
model.eval().to(device)

# Track performance
correct_top1 = 0
correct_top5 = 0
total_images = 0
saved_examples = 0

print("Running FGSM attack on test dataset...")

# Loop through the test dataset
for images, labels in tqdm(test_loader):
    images, labels = images.to(device), labels.to(device)
    images.requires_grad = True

    # Get model predictions
    outputs = model(images)
    true_labels = torch.tensor([index_mapping[int(label)] for label in labels]).to(device)

    # Compute loss and gradients
    loss = F.cross_entropy(outputs, true_labels)
    model.zero_grad()
    loss.backward()

    # Apply FGSM: add perturbation in direction of gradient sign
    perturbation = epsilon * images.grad.sign()
    adversarial_images = torch.clamp(images + perturbation, 0, 1)

    # Evaluate model on adversarial images
    with torch.no_grad():
        adv_outputs = model(adversarial_images)
        _, top5_preds = adv_outputs.topk(5, dim=1)
        top1_preds = top5_preds[:, 0]

        correct_top1 += (top1_preds == true_labels).sum().item()
        correct_top5 += sum([true_labels[i] in top5_preds[i] for i in range(len(true_labels))])
        total_images += labels.size(0)

    # Save first 3–5 adversarial examples
    if saved_examples < 5:
        for i in range(min(5 - saved_examples, len(adversarial_images))):
            img = unnormalize(adversarial_images[i].cpu())
            img = torch.clamp(img, 0, 1)
            img_np = img.permute(1, 2, 0).detach().numpy()
            plt.imshow(img_np)
            plt.title("Adversarial Example")
            plt.axis('off')
            plt.savefig(f"fgsm_example_{saved_examples}.png")
            plt.close()
            saved_examples += 1

# Final accuracy results
top1_acc = 100 * correct_top1 / total_images
top5_acc = 100 * correct_top5 / total_images

print(f"\nTop-1 Accuracy after FGSM attack: {top1_acc:.2f}%")
print(f"Top-5 Accuracy after FGSM attack: {top5_acc:.2f}%")


Running FGSM attack on test dataset...


100%|██████████| 16/16 [00:05<00:00,  2.89it/s]


Top-1 Accuracy after FGSM attack: 26.40%
Top-5 Accuracy after FGSM attack: 50.60%


In [ ]:
## Task-3

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import json
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

# -------------------------
# Configuration Parameters
# -------------------------
epsilon = 0.02           # Maximum total perturbation
alpha = 0.005            # Step size per iteration
steps = 10               # Number of attack iterations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -------------------------
# Image Preprocessing Setup
# -------------------------
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Used to reverse normalization for visualization
unnormalize = transforms.Normalize(
    mean=[-m/s for m, s in zip(mean, std)],
    std=[1/s for s in std]
)

# -------------------------
# Load Test Dataset
# -------------------------
dataset_path = "/content/drive/MyDrive/TestDataSet"
test_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# -------------------------
# Load Class Index Mapping
# -------------------------
with open("/content/drive/MyDrive/TestDataSet/imagenet_class_index.json", "r") as f:
    class_map = json.load(f)

# Maps synset (folder name) → ImageNet index
synset_to_idx = {v[0]: int(k) for k, v in class_map.items()}
folder_to_idx = test_dataset.class_to_idx
index_mapping = {
    folder_to_idx[synset]: synset_to_idx[synset]
    for synset in folder_to_idx
}

# -------------------------
# Load Pretrained Model
# -------------------------
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
model.eval().to(device)

# -------------------------
# Run I-FGSM Attack
# -------------------------
correct_top1 = 0
correct_top5 = 0
total = 0
saved = 0

print("Running I-FGSM attack...")

for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    true_labels = torch.tensor([index_mapping[int(label)] for label in labels]).to(device)

    # Initialize adversarial image
    adv_images = images.clone().detach().requires_grad_(True)

    for _ in range(steps):
        outputs = model(adv_images)
        loss = F.cross_entropy(outputs, true_labels)

        model.zero_grad()
        loss.backward()

        # Take a small step in direction of gradient
        adv_images = adv_images + alpha * adv_images.grad.sign()

        # Clip to keep within epsilon-ball of original
        delta = torch.clamp(adv_images - images, min=-epsilon, max=epsilon)
        adv_images = torch.clamp(images + delta, min=0, max=1).detach().requires_grad_(True)

    # Evaluate model on adversarial images
    with torch.no_grad():
        predictions = model(adv_images)
        _, top5 = predictions.topk(5, dim=1)
        top1 = top5[:, 0]

        correct_top1 += (top1 == true_labels).sum().item()
        correct_top5 += sum([true_labels[i] in top5[i] for i in range(len(true_labels))])
        total += labels.size(0)

    # Save up to 5 adversarial samples
    if saved < 5:
        for i in range(min(5 - saved, len(adv_images))):
            img = unnormalize(adv_images[i].cpu())
            img = torch.clamp(img, 0, 1)
            np_img = img.permute(1, 2, 0).detach().numpy()
            plt.imshow(np_img)
            plt.title("I-FGSM Adversarial Example")
            plt.axis('off')
            plt.savefig(f"ifgsm_example_{saved}.png")
            plt.close()
            saved += 1

# -------------------------
# Final Accuracy Results
# -------------------------
top1_accuracy = 100 * correct_top1 / total
top5_accuracy = 100 * correct_top5 / total

print(f"\nTop-1 Accuracy after I-FGSM attack: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy after I-FGSM attack: {top5_accuracy:.2f}%")


Running I-FGSM attack...


100%|██████████| 16/16 [00:26<00:00,  1.63s/it]


Top-1 Accuracy after I-FGSM attack: 0.40%
Top-5 Accuracy after I-FGSM attack: 6.60%


In [ ]:
## Task-4

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import json
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Device and Parameters
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
patch_ratio = 0.125  # Patch is 1/8th of the image size
patch_type = "random"  # Options: "random" or "black"

# -------------------------
# Image Preprocessing
# -------------------------
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Used to unnormalize for visualization
unnormalize = transforms.Normalize(
    mean=[-m/s for m, s in zip(mean, std)],
    std=[1/s for s in std]
)

# -------------------------
# Load Dataset
# -------------------------
dataset_path = "/content/drive/MyDrive/TestDataSet"
test_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# -------------------------
# Load Label Mapping
# -------------------------
with open("/content/drive/MyDrive/TestDataSet/imagenet_class_index.json", "r") as f:
    class_map = json.load(f)

# Create mapping from folder name (synset) → ImageNet class index
synset_to_idx = {v[0]: int(k) for k, v in class_map.items()}
folder_to_idx = test_dataset.class_to_idx
index_mapping = {
    folder_to_idx[synset]: synset_to_idx[synset]
    for synset in folder_to_idx
}

# -------------------------
# Load Pretrained Model
# -------------------------
model = torchvision.models.resnet34(weights='IMAGENET1K_V1')
model.eval().to(device)

# -------------------------
# Attack Evaluation
# -------------------------
correct_top1 = 0
correct_top5 = 0
total_images = 0
saved = 0

print("Applying patch attack to images...")

for images, labels in tqdm(test_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Get true ImageNet labels
    true_labels = torch.tensor([index_mapping[int(label)] for label in labels]).to(device)

    # Create a copy of the images to patch
    patched_images = images.clone()

    for i in range(patched_images.size(0)):
        img = patched_images[i]
        c, h, w = img.shape
        ph, pw = int(h * patch_ratio), int(w * patch_ratio)
        y_start, x_start = h - ph, w - pw  # bottom-right corner

        # Choose patch type
        if patch_type == "random":
            patch = torch.rand((c, ph, pw), device=device)
        elif patch_type == "black":
            patch = torch.zeros((c, ph, pw), device=device)

        # Apply patch
        img[:, y_start:y_start+ph, x_start:x_start+pw] = patch

    # Evaluate model on patched images
    with torch.no_grad():
        outputs = model(patched_images)
        _, top5_preds = outputs.topk(5, dim=1)
        top1_preds = top5_preds[:, 0]

        correct_top1 += (top1_preds == true_labels).sum().item()
        correct_top5 += sum([true_labels[i] in top5_preds[i] for i in range(len(true_labels))])
        total_images += labels.size(0)

    # Save a few patched examples
    if saved < 5:
        for i in range(min(5 - saved, len(patched_images))):
            img = unnormalize(patched_images[i].cpu())
            img = torch.clamp(img, 0, 1)
            np_img = img.permute(1, 2, 0).detach().numpy()
            plt.imshow(np_img)
            plt.title("Patched Image")
            plt.axis("off")
            plt.savefig(f"patch_example_{saved}.png")
            plt.close()
            saved += 1

# -------------------------
# Final Accuracy Report
# -------------------------
top1_acc = 100 * correct_top1 / total_images
top5_acc = 100 * correct_top5 / total_images

print(f"\nTop-1 Accuracy after patch attack: {top1_acc:.2f}%")
print(f"Top-5 Accuracy after patch attack: {top5_acc:.2f}%")


Applying patch attack to images...


100%|██████████| 16/16 [00:03<00:00,  4.63it/s]


Top-1 Accuracy after patch attack: 74.60%
Top-5 Accuracy after patch attack: 94.80%


In [ ]:
## Task-5

In [2]:
import pandas as pd

# Define the accuracy results
data = {
    "Attack Type": ["Clean", "FGSM", "I-FGSM", "Patch"],
    "Top-1 Accuracy": ["76.00%", "26.40%", "0.40%", "74.60%"],
    "Top-5 Accuracy": ["94.20%", "50.60%", "6.60%", "94.80%"]
}

# Create and display the DataFrame
df = pd.DataFrame(data)
print("Final Accuracy Comparison (Top-1 and Top-5):")
display(df)


Final Accuracy Comparison (Top-1 and Top-5):


,Attack Type,Top-1 Accuracy,Top-5 Accuracy
0,Clean,76.00%,94.20%
1,FGSM,26.40%,50.60%
2,I-FGSM,0.40%,6.60%
3,Patch,74.60%,94.80%
